## 9. Minimal working example

Below is a minimal example on performing a short MD simulation on a the NVT ensemple, using one-component Lennard-Jones. 

In [ ]:
import hoomd
from hoomd import md
hoomd.context.initialize('--mode=cpu');

In [ ]:
import numpy as np
from matplotlib import pyplot 
import ase # Atomic simulation environment
import ase.io
from ase.visualize import view

### 1. Set up the system

Creating a 10x10x10 simple cubic lattice of particles A & B.

In [ ]:
initial_cell = hoomd.lattice.unitcell(N = 2, # two atoms in the initial cell
                            a1 = [3,0,0],
                            a2 = [0,3,0],
                            a3 = [0,0,3], # these define a cubic box of length 3.0
                            position = [[0.0,0.0,0.0], [1.5, 1.5, 1.5]], # set positions for the two atoms (they should not be placed on top of each other)
                            type_name = ['A', 'B'], # one of the atoms will be called type 'A', the other type 'B'
                            mass = [1.0, 1.0], # 
                            );

In [ ]:
system=hoomd.init.create_lattice(initial_cell,n=10) 

### 2. Cell Visualisation

The intial configuration of out 10x10x10 cell. 

In [ ]:
def save_config(hoomd_system,atom_types=["C","F"]):
    
    lattice=np.array([hoomd_system.box.get_lattice_vector(i=i) for i in range(3)])
    x2 = int(lattice[[0],[0]]) / 2
    y2 = int(lattice[[1],[1]]) / 2
    z2 = int(lattice[[2],[2]]) / 2

    ase_atoms=ase.Atoms(pbc=[(True,True,True)],cell=lattice)
    
    for i in range(system.particles.types.pdata.getN()):
        i_type = system.particles.types.pdata.getType(i)
        i_pos = hoomd_system.particles.pdata.getPosition(i)
        i_pos.x = i_pos.x + x2 ; i_pos.y = i_pos.y + y2 ; i_pos.z = i_pos.z + z2
        ase_atoms.append(ase.Atom(atom_types[i_type],position=[i_pos.x,i_pos.y,i_pos.z]))
    return ase_atoms

In [ ]:
current_config = save_config(system)
ase.io.write("system.pdb", current_config)

In [ ]:
import pytraj as pt
import nglview as nv
p_traj = pt.load('system.pdb')
p_view = nv.show_pytraj(p_traj)
p_view.add_unitcell()
p_view

### 3. Define interaction between particles
Setting the cut off parameter as well as  $\sigma$ and $\epsilon$.

In [ ]:
# Specify Lennard-Jones interactions between particle pairs

nl = md.nlist.cell() # atomic distances will have to be calculated first

lj = md.pair.lj(r_cut=2.5, nlist=nl)# define potential

lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0)
lj.pair_coeff.set('A', 'B', epsilon=0.6, sigma=0.9) # set AB epsilon smaller than either AA or BB epsilon. 
lj.pair_coeff.set('B', 'B', epsilon=0.9, sigma=0.8)


### 4. Set up the MD simulation and the ensemble

In [ ]:
all = hoomd.group.all() # all particles are to be treated the same during the molecular dynamics steps

md.integrate.mode_standard(dt=0.005) # the length of one MD timestep the integrator will use

In [ ]:
# Integrate at constant temperature
nvt = hoomd.md.integrate.langevin(group=hoomd.group.all(), kT=1.0, seed=4)
hoomd.run(1000) #run for the first 1000 time steps in NVT
nvt.disable()

# Integrate at constant pressure
npt = hoomd.md.integrate.npt(group=all, kT=0.8, tau=3.0, P=0.1, tauP = 3.0, couple="xyz")
hoomd.run(1000) #run the next 1000 time steps in NPT

Parameters of the integrator can be changed between runs, e.g.: (Don't forget, that on changing the parameters you will need re-equilibrate the system!)

In [ ]:
#nvt.set_params(kT=0.6)
npt.set_params(P=0.8)

### 5. Visualising the equilibriation

In [ ]:
quantities=['volume','lx','ly','lz','potential_energy','kinetic_energy','temperature','pressure']

In [ ]:
hoomd.analyze.log(filename="log-output.4.log",
                  quantities=quantities,
                  period=100,
                  overwrite=True) # keep appending the existing file

In [ ]:
d = hoomd.dump.dcd("dump.dcd", period=200, group=all, overwrite=True, unwrap_full=False);

In [ ]:
hoomd.run(60000)

In [ ]:
current_config = save_config(system)
ase.io.write("system.extxyz", current_config, format="extxyz")
ase.io.write("system.pdb", current_config)

In [ ]:
p_traj = pt.load('dump.dcd', top='system.pdb')
p_view = nv.show_pytraj(p_traj)
p_view

### 8. Analysing results

In [ ]:
data = np.genfromtxt(fname='log-output.4.log', skip_header=True)
# Remember:
# quantities=['volume','lx','ly','lz','potential_energy','kinetic_energy','temperature','pressure']

In [ ]:
pyplot.figure(figsize=(8,4), dpi=200)
pyplot.plot(data[:,1])
pyplot.xlabel('time step')
pyplot.ylabel('volume')

In [ ]:
# Run for 10,000 time steps
#hoomd.run(10e3)